In [9]:
import requests
import pandas as pd
from config import cid
from config import secret
import numpy as np
import json
import csv
import os
import base64
import pprint
import spotipy
pp = pprint.PrettyPrinter(indent=4)

In [22]:
CLIENT_ID = cid
client_secret =  os.getenv('SPOTIFY_SECRET',secret)
B64AUTH = base64.b64encode(f"{CLIENT_ID}:{client_secret}".encode('utf8')).decode('utf8')
baseURL = "https://accounts.spotify.com"

In [23]:
def refresh(refresh_token):
    token = requests.post(f"{baseURL}/api/token", 
                            data={'grant_type':'refresh_token',
                                  'refresh_token':refresh_token},
                            headers={'content_type':'application/x-www-form-urlencoded',
                                    'Authorization':f'Basic {B64AUTH}'})
    return token.json()

In [24]:
refreshtoken="AQCp8g9sp3ayzr-AGzr9xbQfanM5JXrCFdlWW9cDcI9OGL1Q9hciS3Juen4Zi7qAz4BT5TT6OzDQ8JG81E1nM5S4Nfj0m4_0r6TDZDw-J55RzQCt5_LY6dhOplRvw72QhT4"
print(refresh(refreshtoken))

{'access_token': 'BQBdNmBS4sF-E0SDt_xlSf-w87q9rsuh71tHLgUGh7Cvzn2FlDV8jer-3SRKYwFl-d7mu4mND3rlNmGUUmFULLuKSanYWQDyg4CJ28GuXq-uQ5unVtfQ4o6btTp_IjvOaYJaTJKnUgXRrB3Z_vdiYbgoloxT3aVYXUsWR8GEovCKJYg', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': 'user-read-private'}


In [25]:
#Get a list of categories
token=refresh(refreshtoken)["access_token"]
URL="https://api.spotify.com/v1/browse/categories"
params={"limit":5}
response=requests.get(URL,params=params,headers={"Authorization": f"Bearer {token}"})
res=response.json()

In [17]:
sp = spotipy.Spotify(auth = token)

genres = 'blues classical country disco hiphop jazz metal pop reggae rock electronic'.split()

playlists = '''7qACZGMjyo64TdUdKAegjp 3HYK6ri0GkvRcM6GkKh0hJ 
            4mijVkpSXJziPiOrK7YX4M 0ZVSWcJIf7cvycEn9HUvps 6MXkE0uYF4XwU4VTtyrpfP 
            5EyFMotmvSfDAZ4hSdKrbx 3pBfUFu8MkyiCYyZe849Ks 6gS3HhOiI17QNojjPuPzqc 
            0TcXdt4sbITbwCwwFbKYyd 7dowgSWOmvdpwNkGFMUs6e 6I0NsYzfoj7yHXyvkZYoRx'''.split()

columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 
           'acousticness', 'instrumentalness','liveness', 'valence', 'tempo', 'genre']

to_append = []

for g in zip(genres, playlists):
    
    # get playlist for given genre (default limit is 100)
    track_ids = sp.user_playlist_tracks('thesoundsofspotify', playlist_id=g[1], 
                                        fields='items(track(id))')
    # some lists for gathering data 
    tracks_list = []
    
    for track in track_ids['items']:
        tracks_list.append(track['track']['id'])
    
    # get audio features for 50 tracks at a time (spotipy only allows 50 at once)
    # code below can be looped if more tracks need to be analyzed
    tracks_af = sp.audio_features(tracks_list[:50])
    for track_num in range(0, 50):
        track_data = []
        track_data.append(tracks_af[track_num]['danceability'])
        track_data.append(tracks_af[track_num]['energy'])
        track_data.append(tracks_af[track_num]['key'])
        track_data.append(tracks_af[track_num]['loudness'])
        track_data.append(tracks_af[track_num]['mode'])
        track_data.append(tracks_af[track_num]['speechiness'])
        track_data.append(tracks_af[track_num]['acousticness'])
        track_data.append(tracks_af[track_num]['instrumentalness'])
        track_data.append(tracks_af[track_num]['liveness'])
        track_data.append(tracks_af[track_num]['valence'])
        track_data.append(tracks_af[track_num]['tempo'])
        track_data.append(g[0])
        to_append.append(track_data)
audio_data = pd.DataFrame(to_append, columns=columns)

In [18]:
audio_data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,0.705,0.483,1,-12.209,1,0.0542,0.15000,0.026500,0.2020,0.713,122.863,blues
1,0.633,0.625,0,-10.951,1,0.0436,0.49200,0.000418,0.2040,0.841,103.355,blues
2,0.728,0.054,11,-22.557,0,0.0380,0.91000,0.048000,0.1200,0.425,78.033,blues
3,0.582,0.302,9,-11.624,1,0.0353,0.94700,0.010700,0.0931,0.642,94.288,blues
4,0.745,0.436,5,-9.617,0,0.0430,0.59700,0.021300,0.6100,0.771,114.216,blues
...,...,...,...,...,...,...,...,...,...,...,...,...
545,0.684,0.684,1,-13.337,1,0.0418,0.38500,0.290000,0.1060,0.616,113.988,electronic
546,0.711,0.927,2,-8.194,1,0.0374,0.00046,0.832000,0.0506,0.460,132.029,electronic
547,0.510,0.434,1,-10.004,1,0.0361,0.17700,0.516000,0.0603,0.207,119.695,electronic
548,0.783,0.411,8,-19.591,0,0.0870,0.09730,0.697000,0.0599,0.850,100.897,electronic


In [19]:
audio_data.to_csv('audiodata.csv', encoding='utf-8')

In [27]:
sp.current_user_top_tracks

<bound method Spotify.current_user_top_tracks of <spotipy.client.Spotify object at 0x000001D8D5B97648>>

In [28]:
results = sp.current_user_top_tracks(limit=50, offset=0,time_range='medium_term')

HTTP Error for GET to https://api.spotify.com/v1/me/top/tracks returned 401 due to The access token expired


SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/me/top/tracks?time_range=medium_term&limit=50&offset=0:
 The access token expired